In [16]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn as nn
import torch.nn.functional as F
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
clf = pipeline('sentiment-analysis')
result = clf('what a beautiful day!')[0]
result['label']

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

c:\Users\admin\anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\admin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

'POSITIVE'

In [3]:
result['label'], result['score']

('POSITIVE', 0.9998812675476074)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [6]:
input_sentence = "She angered me with her inappropriate comments, rumor-spreading, and disrespectfulness at the formal dinner table"
target_sequence = "She made me angry when she was rude at dinner"

In [7]:
tokens = tokenizer(input_sentence, target_sequence, return_tensors="pt")
logits = model(**tokens).logits

In [8]:
results = torch.softmax(logits, dim=1).tolist()[0]
results

[0.29214248061180115, 0.7078575491905212]

In [9]:
for i, label in enumerate(['no', 'yes']):
    print(f"{label}: {int(round(results[i] * 100))}%")

no: 29%
yes: 71%


In [13]:
fileids = movie_reviews.fileids()
reviews = [movie_reviews.raw(fileid) for fileid in fileids]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids] 

In [14]:
label_dict = {'pos':1, 'neg':0}
y = np.array([label_dict[c] for c in categories])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)
len(X_train), len(X_test)

(1600, 400)

In [17]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [18]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = model.to(device)

In [21]:
batch_size = 10
y_pred = []
num_batch = len(y_test) // batch_size

In [22]:
for i in range(num_batch):
    inputs = tokenizer(X_test[i*batch_size:(i+1)*batch_size], truncation=True, padding=True, return_tensors="pt")
    inputs = inputs.to(device)
    logits = model(**inputs).logits
    pred = F.softmax(logits, dim=-1)
    results = pred.cpu().detach().numpy().argmax(axis=1)
    
    y_pred.extend(results.tolist())

torch.cuda.empty_cache()

In [23]:
score = sum(y_test == np.array(y_pred)) / len(y_test)
score

0.8425